# Proyecto Final  

Pablo Izquierdo Conde

Jorge Pose Eiroa

In [1]:
import wikipediaapi as wk

wiki = wk.Wikipedia('en')

# Cogemos las categorías seleccionadas
cat1 = wiki.page("Category:Esports")
cat2 = wiki.page("Category:Combat_sports")

In [2]:
# Función que busca recursivamente en las categorías y nos devuelve una lista con las páginas
def print_categorymembers(categorymembers, level=0, max_level=2, pages = []):
    for c in categorymembers.values():
        if c.ns == 0: # Sacamos las páginas (ns=0)
            pages.append(c)
            print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wk.Namespace.CATEGORY and level <= max_level:
            print_categorymembers(c.categorymembers, level + 1,1, pages)
p1 = []
p2 = []
print_categorymembers(cat1.categorymembers,0,1, p1)
print_categorymembers(cat2.categorymembers,0,1, p2)

Hacemos una permutacion de los indices de forma aleatoria, para coger solo 1000 documentos.

In [27]:
import numpy as np

indices1 = np.random.permutation(len(p1))[:1000]
indices2 = np.random.permutation(len(p2))[:1000]

#print(indices1)

In [28]:
# Sacamos el texto de las listas de páginas
corpus0 = []
corpus1 = []

for n,i in enumerate(indices1):
    if not n%100:
        print('\rPage', n, 'out of', 1000, end='', flush=True)
    corpus0.append(p1[i].text)
    
for n,i in enumerate(indices2):
    if not n%100:
        print('\rPage', n, 'out of', 1000, end='', flush=True)
    corpus1.append(p2[i].text)

Page 900 out of 4290

In [151]:
print(len(corpus0))
print(len(corpus1))

1000
1000


# 2. Procesado de texto

    1. Tokenization
    2. Homogeneization
    3. Cleaning
    4. Vectorization

In [79]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wnl = WordNetLemmatizer()
stopwords_en = stopwords.words('english')

In [81]:
palabros = ['wa','also','ha']

stopwords_en.extend(palabros)

print(stopwords_en)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [152]:
def getCorpusClean(corpus):
        
    return corpus_clean

corpus0_clean = []
corpus1_clean = []

for text in corpus0:
    tokens = word_tokenize(text)
    tokens_filtered = [el.lower() for el in tokens if el.isalnum() and not el.isdigit()] # Nos quedamos solo con palabras, quitamos numeros
    tk_lemmat = [wnl.lemmatize(el) for el in tokens_filtered]
    tk_clean = [tk for tk in tk_lemmat if tk not in stopwords_en]
    corpus0_clean.append(tk_clean)

for text in corpus1:
    tokens = word_tokenize(text)
    tokens_filtered = [el.lower() for el in tokens if el.isalnum() and not el.isdigit()] # Nos quedamos solo con palabras, quitamos numeros
    tk_lemmat = [wnl.lemmatize(el) for el in tokens_filtered]
    tk_clean = [tk for tk in tk_lemmat if tk not in stopwords_en]
    corpus1_clean.append(tk_clean)



In [154]:
print(len(corpus0_clean))
print(len(corpus1_clean))

1000
1000


In [155]:
import gensim

def getCorpusBow(corpus_clean, no_below = 5, no_above = .75):
    D = gensim.corpora.Dictionary(corpus_clean)
    D.filter_extremes(no_below=no_below, no_above=no_above, keep_n=25000)
    corpus_bow = [D.doc2bow(doc) for doc in corpus_clean]
    return corpus_bow, D
    
corpus0_bow, D0 = getCorpusBow(corpus0_clean)
corpus1_bow, D1 = getCorpusBow(corpus1_clean)

# 3. Modelado de tópicos con LDA

In [157]:
def getLDAModel(corpus_bow, D, num_topics):
    ldag = gensim.models.ldamodel.LdaModel(corpus=corpus_bow, id2word=D, num_topics = num_topics)
    return ldag

num_topics = 50
ldag0 = getLDAModel(corpus0_bow, D0, num_topics)
ldag1 = getLDAModel(corpus1_bow, D1, num_topics)

In [130]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

def visTopics(ldag, corpus_bow, D):    
    vis_data = gensimvis.prepare(ldag, corpus_bow, D)
    return pyLDAvis.display(vis_data)   

In [131]:
visTopics(ldag0, corpus0_bow, D0)

C:\Users\pablo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
visTopics(ldag1, corpus1_bow, D1)

In [158]:
def getExpandedMatrix(corpus_bow, ldag, num_topics):
        reduced_corpus = [el for el in ldag[corpus_bow[:]]]
        X = gensim.matutils.corpus2dense(reduced_corpus, num_topics).T
        return X

X0 = getExpandedMatrix(corpus0_bow, ldag0, num_topics)
y0 = np.zeros((1000,1))
X1 = getExpandedMatrix(corpus1_bow, ldag1, num_topics)
y1 = np.ones((1000,1))

print(X0.shape, X1.shape)

(1000, 50) (1000, 50)


In [159]:
Xtrain = np.vstack((X0, X1))
ytrain = np.vstack((y0,y1))

print(Xtrain.shape)
print(ytrain.shape)

(2000, 50)
(2000, 1)
